In [23]:
import pyomo.environ as pyo
import pyomo.opt as pyopt
from pyomo.core import Suffix

def read_cor_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    num_servers = 5
    num_clients = 25
    fixed_costs = [0] * num_servers
    revenues = {}
    capacity = 188

    for line in lines:
        parts = line.split()
        if len(parts) >= 3:
            if parts[0].startswith('x_') and parts[1] == 'obj':
                server = int(parts[0].split('_')[1]) - 1  # Adjust for 0-based indexing
                fixed_costs[server] = float(parts[2])
            elif parts[0].startswith('x_') and parts[1] == 'c':
                capacity = float(parts[2])
            elif parts[0].startswith('y_') and parts[1] == 'obj':
                client, server = map(int, parts[0].split('_')[1:3])
                client -= 1  # Adjust for 0-based indexing
                server -= 1  # Adjust for 0-based indexing
                revenue = -float(parts[2])  # Negate because we're maximizing
                revenues[(client, server)] = revenue

    print(f"Number of fixed costs: {len(fixed_costs)}")
    print(f"Number of revenue entries: {len(revenues)}")
    print(f"Capacity: {capacity}")
    print(f"Fixed costs: {fixed_costs}")
    print(f"First few revenue entries: {list(revenues.items())[:5]}")

    return num_servers, num_clients, fixed_costs, revenues, capacity

def read_sto_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    scenarios = {}
    current_scenario = None
    scenario_prob = None

    for line in lines:
        parts = line.split()
        if len(parts) >= 4 and parts[0] == 'SC':
            current_scenario = parts[1]
            scenario_prob = float(parts[3])
            scenarios[current_scenario] = {'prob': scenario_prob, 'clients': {}}
        elif len(parts) == 4 and parts[0] == 'RHS':
            client = int(parts[1][1:]) - 7  # c7 corresponds to client 0
            availability = int(parts[3])
            scenarios[current_scenario]['clients'][client] = availability

    # Print some debug information
    print(f"Number of scenarios: {len(scenarios)}")
    if scenarios:
        first_scenario = next(iter(scenarios.values()))
        print(f"Number of clients in first scenario: {len(first_scenario['clients'])}")
        print(f"Sample of first scenario data: {list(first_scenario['clients'].items())[:5]}")

    return scenarios


def create_model(num_servers, num_clients, fixed_costs, revenues, capacity, scenarios):
    model = pyo.ConcreteModel()

    # Sets
    model.I = pyo.RangeSet(0, num_clients-1)  # Clients
    model.J = pyo.RangeSet(0, num_servers-1)  # Servers
    model.S = pyo.Set(initialize=scenarios.keys())  # Scenarios

    # First-stage variables
    model.x = pyo.Var(model.J, domain=pyo.Binary)

    # Second-stage variables
    model.y = pyo.Var(model.I, model.J, model.S, domain=pyo.Binary)

    # Objective function
    def obj_rule(model):
        first_stage = sum(fixed_costs[j] * model.x[j] for j in model.J)
        second_stage = sum(scenarios[s]['prob'] * sum(revenues.get((i,j), 0) * model.y[i,j,s] 
                           for i in model.I for j in model.J) for s in model.S)
        return first_stage - second_stage  # Minimize costs - revenues

    model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

    # Constraints
    def capacity_rule(model, j, s):
        return sum(model.y[i,j,s] for i in model.I) <= capacity * model.x[j]

    model.capacity = pyo.Constraint(model.J, model.S, rule=capacity_rule)

    def assignment_rule(model, i, s):
        return sum(model.y[i,j,s] for j in model.J) <= scenarios[s]['clients'].get(i, 0)

    model.assignment = pyo.Constraint(model.I, model.S, rule=assignment_rule)

    # Maximum number of servers
    model.max_servers = pyo.Constraint(expr=sum(model.x[j] for j in model.J) <= 5)

    return model

def solve_model(model):
    solver = pyopt.SolverFactory('glpk')
    results = solver.solve(model, tee=True)

    if results.solver.status == pyopt.SolverStatus.ok and results.solver.termination_condition == pyopt.TerminationCondition.optimal:
        print("Optimal solution found")
        print("Opened servers:")
        for j in model.J:
            if pyo.value(model.x[j]) > 0.5:
                print(f"Server {j+1}")
        
        print("\nObjective value:", pyo.value(model.obj))
    else:
        print("Solver did not find an optimal solution")

def main():
    cor_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.cor'
    sto_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.sto'

    num_servers, num_clients, fixed_costs, revenues, capacity = read_cor_file(cor_file_path)
    scenarios = read_sto_file(sto_file_path)

    model = create_model(num_servers, num_clients, fixed_costs, revenues, capacity, scenarios)
    solve_model(model)

if __name__ == "__main__":
    main()
 # cor_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.cor'
    # sto_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.sto'


Number of fixed costs: 5
Number of revenue entries: 121
Capacity: 188
Fixed costs: [1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
First few revenue entries: [((0, 1), 22.0), ((0, 2), 18.0), ((0, 3), 14.0), ((0, 4), 22.0), ((1, 0), 15.0)]
Number of scenarios: 50
Number of clients in first scenario: 0
Sample of first scenario data: []
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/tmp1cgh5j1b.glpk.raw
 --wglp /var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/tmpyy7gn3li.glpk.glp
 --cpxlp /var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/tmp10f0br43.pyomo.lp
Reading problem data from '/var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/tmp10f0br43.pyomo.lp'...
/var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/tmp10f0br43.pyomo.lp:29578: warning: lower bound of variable 'x2' redefined
/var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/tmp10f0br43.pyomo.lp:29578: warning: upper bound of variable 'x2' redefined
1501 rows

# Experiment section

In [24]:
cor_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.cor'
sto_file_path = '/Users/vivekchaudhary/Documents/progressive_hedging/sslp/sslp_5_25_50.sto'

num_servers, num_clients, fixed_costs, revenues, capacity = read_cor_file(cor_file_path)
scenarios = read_sto_file(sto_file_path)

Number of fixed costs: 5
Number of revenue entries: 121
Capacity: 188
Fixed costs: [1000.0, 1000.0, 1000.0, 1000.0, 1000.0]
First few revenue entries: [((0, 1), 22.0), ((0, 2), 18.0), ((0, 3), 14.0), ((0, 4), 22.0), ((1, 0), 15.0)]
Number of scenarios: 50
Number of clients in first scenario: 0
Sample of first scenario data: []


In [25]:
scenarios

{'Scen1': {'prob': 0.02, 'clients': {}},
 'Scen2': {'prob': 0.02, 'clients': {}},
 'Scen3': {'prob': 0.02, 'clients': {}},
 'Scen4': {'prob': 0.02, 'clients': {}},
 'Scen5': {'prob': 0.02, 'clients': {}},
 'Scen6': {'prob': 0.02, 'clients': {}},
 'Scen7': {'prob': 0.02, 'clients': {}},
 'Scen8': {'prob': 0.02, 'clients': {}},
 'Scen9': {'prob': 0.02, 'clients': {}},
 'Scen10': {'prob': 0.02, 'clients': {}},
 'Scen11': {'prob': 0.02, 'clients': {}},
 'Scen12': {'prob': 0.02, 'clients': {}},
 'Scen13': {'prob': 0.02, 'clients': {}},
 'Scen14': {'prob': 0.02, 'clients': {}},
 'Scen15': {'prob': 0.02, 'clients': {}},
 'Scen16': {'prob': 0.02, 'clients': {}},
 'Scen17': {'prob': 0.02, 'clients': {}},
 'Scen18': {'prob': 0.02, 'clients': {}},
 'Scen19': {'prob': 0.02, 'clients': {}},
 'Scen20': {'prob': 0.02, 'clients': {}},
 'Scen21': {'prob': 0.02, 'clients': {}},
 'Scen22': {'prob': 0.02, 'clients': {}},
 'Scen23': {'prob': 0.02, 'clients': {}},
 'Scen24': {'prob': 0.02, 'clients': {}},
 